In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:40109")
client

<Client: 'tcp://127.0.0.1:40109' processes=10 threads=10, memory=1.05 TiB>

In [2]:
import numpy as np
import xarray as xr
import os.path as op
from rechunker import rechunk
import sys
from xgcm.grid import Grid
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
ddir = '/gpfs/research/coaps/tu22/HYCOM50'

In [4]:
pdir = op.join(ddir,'../BB86_PACKAGE/PYTHON/')
sys.path.insert(0, pdir)

from hycom.info import read_field_names, read_field_grid_names
from hycom.io import read_hycom_fields, read_hycom_grid, sub_var2

In [5]:
idm = 6709 
jdm = 7373
kdm = 32
layers = np.arange(kdm)

ychunk = 100
xchunk = 80
lchunk = 2

In [6]:
nF = 25
if nF == 30:
    run = 'ATLb'
else:
    run = 'ATLc'
file = '%03d_archMNA.0016_0020.a' % nF
file_data = op.join(ddir, '%4s0.02/'%run+file)
print(file_data)

# get metadata of .[ab]
# Printing the fields available in the file
print(F"The fields available are: {read_field_names(file_data)}")

/gpfs/research/coaps/tu22/HYCOM50/ATLc0.02/025_archMNA.0016_0020.a
The fields available are: ['montg1', 'srfhgt', 'surflx', 'salflx', 'bl_dpth', 'mix_dpth', 'tmix', 'smix', 'thmix', 'umix', 'vmix', 'kemix', 'covice', 'thkice', 'temice', 'u_btrop', 'v_btrop', 'kebtrop', 'u-vel.', 'v-vel.', 'k.e.', 'thknss', 'temp', 'salin', 'density']


In [7]:
meta = 'regional.grid.a'
meta_data = op.join(ddir, '%4s0.02/topo/'%run+meta)
print(meta_data)

/gpfs/research/coaps/tu22/HYCOM50/ATLc0.02/topo/regional.grid.a


In [8]:
ds = xr.open_zarr(op.join(ddir,'%4s0.02/topo/Coords.zarr' % run))
ds

<xarray.Dataset>
Dimensions:  (XC: 6709, XG: 6709, YC: 7373, YG: 7373)
Coordinates:
  * XC       (XC) float64 -98.69 -98.67 -98.65 -98.63 ... 38.9 38.92 38.94 38.96
  * XG       (XG) float64 -98.7 -98.68 -98.66 -98.64 ... 38.89 38.91 38.93 38.95
  * YC       (YC) float64 -27.94 -27.92 -27.91 -27.89 ... 71.34 71.35 71.35
  * YG       (YG) float64 -27.95 -27.93 -27.92 -27.9 ... 71.34 71.34 71.35 71.35
Data variables:
    tlat     (YC, XC) float64 dask.array<chunksize=(100, 80), meta=np.ndarray>
    tlon     (YC, XC) float64 dask.array<chunksize=(100, 80), meta=np.ndarray>
    ulon     (YC, XG) float64 dask.array<chunksize=(100, 80), meta=np.ndarray>
    vlat     (YG, XC) float64 dask.array<chunksize=(100, 80), meta=np.ndarray>

In [8]:
tlon = sub_var2(meta_data, idm, jdm, 1)
tlat = sub_var2(meta_data, idm, jdm, 2)
dsave = xr.DataArray(tlon, dims=['YC','XC'],
                  coords={'YC':np.mean(tlat,1),
                          'XC':np.mean(tlon,0)}
                 ).chunk({'YC':ychunk,'XC':xchunk}).to_dataset(name='tlon')
dsave['tlat'] = xr.DataArray(tlat, dims=['YC','XC'],
                          coords={'YC':dsave.YC, 'XC':dsave.XC}
                         ).chunk({'YC':ychunk,'XC':xchunk})
ulon = sub_var2(meta_data, idm, jdm, 5)
vlat = sub_var2(meta_data, idm, jdm, 8)
dsave['ulon'] = xr.DataArray(ulon, dims=['YC','XG'],
                          coords={'YC':dsave.YC,
                                  'XG':np.mean(ulon,0)}
                         ).chunk({'YC':ychunk,'XG':xchunk})
dsave['vlat'] = xr.DataArray(vlat, dims=['YG','XC'],
                          coords={'YG':np.mean(vlat,1),
                                  'XC':dsave.XC}
                         ).chunk({'YG':ychunk,'XC':xchunk})
dsave.to_zarr(op.join(ddir,'%4s0.02/topo/Coords.zarr' % run),
              mode='w')
dsave.close()

In [10]:
dsave = xr.DataArray(sub_var2(meta_data, idm, jdm, 14), dims=['YC','XG'],
                     coords={'YC':ds.YC, 'XG':ds.XG}
                    ).chunk({'YC':ychunk,'XG':xchunk}).to_dataset(name='DXC')
dsave['DYG'] = xr.DataArray(sub_var2(meta_data, idm, jdm, 15), dims=['YC','XG'],
                            coords={'YC':ds.YC, 'XG':ds.XG}
                           ).chunk({'YC':ychunk,'XG':xchunk})
dsave['DXG'] = xr.DataArray(sub_var2(meta_data, idm, jdm, 16), dims=['YG','XC'],
                            coords={'YG':ds.YG, 'XC':ds.XC}
                           ).chunk({'YG':ychunk,'XC':xchunk})
dsave['DYC'] = xr.DataArray(sub_var2(meta_data, idm, jdm, 17), dims=['YG','XC'],
                            coords={'YG':ds.YG, 'XC':ds.XC}
                           ).chunk({'YG':ychunk,'XC':xchunk})
dsave['DXF'] = xr.DataArray(sub_var2(meta_data, idm, jdm, 12), dims=['YG','XG'],
                            coords={'YG':ds.YG, 'XG':ds.XG}
                           ).chunk({'YG':ychunk,'XG':xchunk})
dsave['DYF'] = xr.DataArray(sub_var2(meta_data, idm, jdm, 13), dims=['YG','XG'],
                            coords={'YG':ds.YG, 'XG':ds.XG}
                           ).chunk({'YG':ychunk,'XG':xchunk})
dsave.to_zarr(op.join(ddir,'%4s0.02/topo/Metrics.zarr' % run),
              mode='w')
dsave.close()

In [9]:
fields = ['u-vel.','v-vel.']
hycom_field = read_hycom_fields(file_data, fields, layers)
dsave = xr.DataArray(hycom_field[fields[0]], dims=['layer','YC','XG'],
                     coords={'layer':np.arange(.5,32.5,1.),
                             'YC':ds.YC, 'XG':ds.XG}
                    ).sel(YC=slice(0,60)).chunk({'layer':lchunk,'YC':ychunk,'XG':xchunk}
                                               ).to_dataset(name='U')
dsave.to_zarr(op.join(ddir,'%4s0.02/%03d/0016-0020/U.zarr' 
                      % (run,nF)),
              mode='w')
dsave.close()
dsave = xr.DataArray(hycom_field[fields[1]], dims=['layer','YG','XC'],
                     coords={'layer':np.arange(.5,32.5,1.),
                             'YG':ds.YG, 'XC':ds.XC}
                    ).sel(YG=slice(0,60)).chunk({'layer':lchunk,'YG':ychunk,'XC':xchunk}
                                               ).to_dataset(name='V')
dsave.to_zarr(op.join(ddir,'%4s0.02/%03d/0016-0020/V.zarr' 
                      % (run,nF)),
              mode='w')
dsave.close()

Hycom version: 20, Experiment: 30
CONTINUE of E026 THAT WAS RUN IN GARNET (01-05)


Dims lon: 6709, lat: 7373
	Reading layers [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] for field u-vel.. Total layers: 32
	Reading layers [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] for field v-vel.. Total layers: 32


In [9]:
fields = ['temp','salin']
layers = np.arange(kdm)
hycom_field = read_hycom_fields(file_data, fields, layers)
dsave = xr.DataArray(hycom_field[fields[0]], dims=['layer','YC','XC'],
                     coords={'layer':np.arange(.5,32.5,1.),
                             'YC':ds.YC, 'XC':ds.XC}
                    ).sel(YC=slice(0,60)).chunk({'layer':lchunk,'YC':ychunk,'XC':xchunk}
                                               ).to_dataset(name='THETA')
dsave.to_zarr(op.join(ddir,'%4s0.02/%03d/0016-0020/T.zarr' 
                      % (run,nF)),
              mode='w')
dsave.close()
dsave = xr.DataArray(hycom_field[fields[1]], dims=['layer','YC','XC'],
                     coords={'layer':np.arange(.5,32.5,1.),
                             'YC':ds.YC, 'XC':ds.XC}
                    ).sel(YC=slice(0,60)).chunk({'layer':lchunk,'YC':ychunk,'XC':xchunk}
                                               ).to_dataset(name='SALT')
dsave.to_zarr(op.join(ddir,'%4s0.02/%03d/0016-0020/S.zarr' 
                      % (run,nF)),
              mode='w')
dsave.close()

Hycom version: 20, Experiment: 30
CONTINUE of E026 THAT WAS RUN IN GARNET (01-05)


Dims lon: 6709, lat: 7373
	Reading layers [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] for field temp. Total layers: 32
	Reading layers [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] for field salin. Total layers: 32


In [9]:
fields = ['thknss']
hycom_field= read_hycom_fields(file_data, fields, layers)
dp = xr.DataArray(hycom_field[fields[0]], dims=['layer','YC','XC'],
                  coords={'layer':np.arange(.5,32.5,1.),
                          'YC':ds.YC, 'XC':ds.XC}
                 ).sel(YC=slice(0,60)).chunk({'layer':lchunk,'YC':ychunk,'XC':xchunk})
rho0 = 1000.                                  ## reference density
g = 9.806                                     ## gravity
dsave = (dp/(rho0*g)).to_dataset(name='dZp1') ## convert from pressure to m
# dsave['dZ'] = xr.concat([xr.concat([ds.dZ.isel(layer=0)*.5,
#                     xr.DataArray(ds.dZ.rolling(layer=2).mean().isel(layer=slice(1,None)).data,
#                                  dims=['layer1','YC','XC'],
#                                  coords={'YC':ds.YC,'XC':ds.XC})
#                                  ], 'layer1'), ds.dZ.isel(layer=-1)*.5
#                       ], 'layer1'
#                      ).chunk({'layer1':3})
# dsave.coords['layer1'] = np.arange(33)
dsave.to_zarr(op.join(ddir,'%4s0.02/%03d/0016-0020/dZp1.zarr'
                      % (run,nF)),
              mode='w')
dsave.close()

Hycom version: 20, Experiment: 25
TWIN of ATLc0.02 026 with NewEnglandSeamounts removed


Dims lon: 6709, lat: 7373
	Reading layers [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] for field thknss. Total layers: 32


In [10]:
dsZ = xr.open_zarr(op.join(ddir,'%4s0.02/%03d/0016-0020/dZp1.zarr' 
                           % (run,nF))
                  )
dZ = xr.concat([
                xr.concat([
                    xr.DataArray(dsZp1.dZp1.isel(layer=0).data[np.newaxis],
                                 dims=['layerp1','YC','XC'],
                                 coords={'layerp1':np.array([0]),
                                         'YC':dsZ.YC,'XC':dsZ.XC}
                                )*.5,
                    xr.DataArray(dsZp1.dZp1.rolling(layer=2).mean().isel(layer=slice(1,None)).data,
                                 dims=['layerp1','YC','XC'],
                                 coords={'layerp1':np.arange(1,32),
                                         'YC':dsZ.YC,'XC':dsZ.XC})
                          ], 'layerp1'
                         ).sortby('layerp1'),
                xr.DataArray(dsZp1.dZp1.isel(layer=-1).data[np.newaxis],
                             dims=['layerp1','YC','XC'],
                             coords={'layerp1':np.array([32]),
                                     'YC':dsZ.YC,'XC':dsZ.XC}
                            )*.5
               ], 'layerp1'
              ).sortby('layerp1').compute().chunk({'layerp1':3,'YC':ychunk,'XC':xchunk})
dZ.to_dataset(name='dZ').to_zarr(op.join(ddir,'%4s0.02/%03d/0016-0020/dZ.zarr' 
                                         % (run,nF))
                                )

ValueError: conflicting sizes for dimension 'YC': length 4121 on the data but length 7373 on coordinate 'YC'